In [ ]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings

In [ ]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500.head()

In [ ]:
sp500['Symbol'].value_counts()

In [ ]:
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')
symbols_list = sp500['Symbol'].unique().tolist()

In [ ]:
end_date = '2023-09-27'

start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

data = yf.download(tickers=symbols_list,start=start_date,end=end_date).stack()
data.index.names = ['date','ticker']
data.columns = data.columns.str.lower()
data

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

## Garmman Klass Volatility

In [ ]:
data['german_klass_vol'] = ((np.log(data['high'])-np.log(data['low']))**2)/2-(2*np.log(2)-1)*((np.log(data['adj close'])-np.log(data['open']))**2)

data['rsi'] = data.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x,length=20))

data['bb_low'] = data.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x),length=20).iloc[:,0])

data['bb_mid'] = data.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
data['bb_high'] = data.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

In [ ]:
def compute_atr(stock_data):
  atr = pandas_ta.atr(high=stock_data['high'],low=stock_data['low'],close=stock_data['close'],length=14)
  
  return atr.sub(atr.mean()).div(atr.std())

In [ ]:
data['atr'] = data.groupby(level=1,group_keys=False).apply(compute_atr)

In [ ]:
def compute_macd(close):
  macd = pandas_ta.macd(close=close,length=20).iloc[:,0]
  return macd.sub(macd.mean()).div(macd.std())

data['macd'] = data.groupby(level=1,group_keys=False)['adj close'].apply(compute_macd)

data['dollar_volume'] = (data['adj close'] * data['volume']) / 1e6

In [ ]:
data.iloc[900000:]